In [123]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of ChEmbed.modules.simple_rnn failed: Traceback (most recent call last):
  File "/home/patrick/miniconda3/envs/d2l/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/patrick/miniconda3/envs/d2l/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/home/patrick/miniconda3/envs/d2l/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/home/patrick/miniconda3/envs/d2l/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 365, in update_class
    update_instances(old, new)
  File "/home/patrick/miniconda3/envs/d2l/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 323, in update_instances
    object.__setattr__(ref, "__class__", new)
TypeError: __class__ assignment: 'simpleRNN' object layout differs from 'simpleRNN'
]


In [ ]:
import sys

sys.path.append("../../")

from tokenizers import Tokenizer
import sys

import matplotlib.pyplot as plt
import numpy as np
import collections
import torch

from ChEmbed.data import chembldb, datasets, chembed_tokenize, shakespeare_dataset
from ChEmbed.training import trainer
from ChEmbed.modules import simple_rnn
import attr

from ChEmbed import plots, utilities

from transformers.tokenization_utils_fast import PreTrainedTokenizerFast
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import torch
import attrs
from torch import nn, optim


In [125]:
def decode_indices_to_string(encoded_indices: list, idx_to_char_mapping: dict[int, str]):
    decoded = ''.join([idx_to_char_mapping[int(inx)] for inx in encoded_indices])
    return decoded

def encode_string_to_indices(smiles_string: str, char_to_idx_mapping: dict[str, int]):
    encoded = [char_to_idx_mapping[c] for c in smiles_string]
    return encoded

In [ ]:
chembl_raw = chembldb.ChemblDBChemreps()
chembl_smiles = chembl_raw._load_or_download()["canonical_smiles"].to_list()

In [127]:
chembl_smiles[:10]

['Cc1cc(-c2csc(N=C(N)N)n2)cn1C',
 'CC[C@H](C)[C@H](NC(=O)[C@H](CC(C)C)NC(=O)[C@@H](NC(=O)[C@@H](N)CCSC)[C@@H](C)O)C(=O)NCC(=O)N[C@@H](C)C(=O)N[C@@H](C)C(=O)N[C@@H](Cc1c[nH]cn1)C(=O)N[C@@H](CC(N)=O)C(=O)NCC(=O)N[C@@H](C)C(=O)N[C@@H](C)C(=O)N[C@@H](CCC(N)=O)C(=O)N[C@@H](CC(C)C)C(=O)N[C@@H](CC(C)C)C(=O)N[C@@H](CCCN=C(N)N)C(=O)N[C@@H](CCC(N)=O)C(=O)N[C@@H](CC(C)C)C(=O)N[C@@H](CCCN=C(N)N)C(=O)NCC(=O)N[C@@H](CCC(N)=O)C(=O)N[C@@H](CC(C)C)C(=O)NCC(=O)N1CCC[C@H]1C(=O)N1CCC[C@H]1C(=O)NCC(=O)N[C@@H](CO)C(=O)N[C@@H](CCCN=C(N)N)C(N)=O',
 'CCCC[C@@H]1NC(=O)[C@@H](NC(=O)[C@H](CC(C)C)NC(=O)[C@@H](NC(=O)[C@H](CCC(=O)O)NC(=O)[C@H](CCCN=C(N)N)NC(=O)[C@H](CC(C)C)NC(=O)[C@H](CC(C)C)NC(=O)[C@H](Cc2c[nH]cn2)NC(=O)[C@H](N)Cc2ccccc2)C(C)C)CCC(=O)NCCCC[C@@H](C(=O)N[C@@H](CCC(N)=O)C(=O)N[C@@H](CC(C)C)C(=O)N[C@@H](C)C(=O)N[C@@H](CCC(N)=O)C(=O)N[C@@H](CCC(N)=O)C(=O)N[C@@H](C)C(=O)N[C@@H](Cc2c[nH]cn2)C(=O)N[C@@H](CO)C(=O)N[C@@H](CC(N)=O)C(=O)N[C@@H](CCCN=C(N)N)C(=O)N[C@@H](CCCCN)C(=O)N[C@@H](CC(C)C)C(=O)N[C@@H](CCC

In [ ]:
chembl_mini = datasets.CharacterLevelSMILES(
    smiles_list = chembl_smiles[:100000],
    length = 256,
    batch_size = 128
)

In [136]:
len(chembl_mini.all_smiles)

5511270

In [137]:
model = simple_rnn.simpleRNN(
    # Mandatory
    num_hiddens = 512,
    vocab_size = len(chembl_mini.characters),
    # tuning
    learning_rate = 0.001,
    weight_decay = 1e-4,
    num_layers = 3
)

In [138]:
model_trainer = trainer.Trainer(max_epochs=32, init_random=None, clip_grads_norm=1.0)
model_trainer.fit(model, chembl_mini)

Epoch 1/32: Train Loss: 2.1607, Val Loss: 1.1749
Epoch 2/32: Train Loss: 1.0800, Val Loss: 0.9705
Epoch 3/32: Train Loss: 0.9493, Val Loss: 0.9045
Epoch 4/32: Train Loss: 0.8911, Val Loss: 0.8662
Epoch 5/32: Train Loss: 0.8496, Val Loss: 0.8433
Epoch 6/32: Train Loss: 0.8206, Val Loss: 0.8312
Epoch 7/32: Train Loss: 0.7990, Val Loss: 0.8091
Epoch 8/32: Train Loss: 0.7805, Val Loss: 0.7991
Epoch 9/32: Train Loss: 0.7646, Val Loss: 0.7922
Epoch 10/32: Train Loss: 0.7527, Val Loss: 0.7991
Epoch 11/32: Train Loss: 0.7420, Val Loss: 0.7845
Epoch 12/32: Train Loss: 0.7309, Val Loss: 0.7801
Epoch 13/32: Train Loss: 0.7214, Val Loss: 0.7767
Epoch 14/32: Train Loss: 0.7153, Val Loss: 0.7828
Epoch 15/32: Train Loss: 0.7075, Val Loss: 0.7757
Epoch 16/32: Train Loss: 0.6993, Val Loss: 0.7645
Epoch 17/32: Train Loss: 0.6924, Val Loss: 0.7671
Epoch 18/32: Train Loss: 0.6871, Val Loss: 0.7658
Epoch 19/32: Train Loss: 0.6805, Val Loss: 0.7628
Epoch 20/32: Train Loss: 0.6780, Val Loss: 0.7618
Epoch 21/

In [152]:
def simple_generate(prefix, num_chars, model, char_to_idx_mapping, idx_to_char_mapping, temperature=0.0, device=None):
    """
    Simple character-by-character generation function.
    """

    def decode_indices_to_string(encoded_indices: list, idx_to_char_mapping: dict[int, str]):
        decoded = ''.join([idx_to_char_mapping[int(inx)] for inx in encoded_indices])
        return decoded

    def encode_string_to_indices(smiles_string: str, char_to_idx_mapping: dict[str, int]):
        encoded = [char_to_idx_mapping[c] for c in smiles_string]
        return encoded

    model.eval()
    generated = prefix
    
    with torch.no_grad():
        for i in range(num_chars):
            # Encode current text
            encoded = torch.nn.functional.one_hot(torch.tensor(encode_string_to_indices(generated, char_to_idx_mapping)), num_classes=len(char_to_idx_mapping))
            input_tensor = torch.tensor(encoded, device=device, dtype=torch.float32)
            
            # Get prediction
            output = model(input_tensor.unsqueeze(0))  # Add batch dim
            
            # Get most likely next token
            if temperature > 0:
                # Apply temperature scaling
                output = output / temperature
                probabilities = torch.softmax(output, dim=-1)
                next_token = torch.multinomial(probabilities[0, -1, :], num_samples=1).item()
            else:
                # Default to argmax if temperature is 0
                next_token = output[0, -1, :].argmax().item()
            
            # Decode and append
            next_char = decode_indices_to_string([next_token], idx_to_char_mapping)
            generated += next_char
            
            # print(f"Step {i+1}: Added '{next_char}' -> '{generated}'")
            
    return generated

In [ ]:
# test by making some molecules
print(simple_generate("C=O", 200, model, chembl_mini.char_to_idx, chembl_mini.idx_to_char, temperature=0.5, device='cuda').replace(" ", "\n"))

/tmp/ipykernel_122935/2233264071.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_tensor = torch.tensor(encoded, device=device, dtype=torch.float32)


C=O)c1ccccc1
CCCCN1C(=O)c2ccccc2S(=O)(=O)C1
CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC(=O)OCC
CC(C)(C)NC(=O)Nc1cccc(OCc2ccccc2)c1
CCOC(=O)C(=O)Nc1cccc(C(=O)NCCc2ccccc2)c1
CCCCCCCCCCCCCCCCOCC(O)COC(CO)C(O)COC
CC
